# Model

> All things related to models (loading, utils,...)

In [ ]:
#| default_exp model

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os

import torch
import torch.nn as nn
from torchvision import transforms, models



In [ ]:
#| export
def build_model(backbone="resnet18", num_classes=3, pretrained=True):

    if backbone == "resnet18":
        model = models.resnet18(pretrained=pretrained)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif backbone == "efficientnet":
        model = models.efficientnet_b0(pretrained=pretrained)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError("Unsupported backbone")
    return model

In [ ]:
#| export
def load_ckpt(model, ckpt_path):
    if os.path.exists(ckpt_path):
        state_dict = torch.load(ckpt_path, map_location='cpu')
        model.load_state_dict(state_dict)
        print(f"Loaded pretrained weights from {ckpt_path}")
    else:
        print(f"No checkpoint found at {ckpt_path}. Using random initialized weights.")
    return model

In [ ]:
#| export
def init_model(cfg): 
    
    model = build_model(backbone=cfg.model.backbone, num_classes=len(cfg.data.label_names), pretrained=cfg.model.pretrained)
    model = load_ckpt(model, cfg.model.ckpt)
    for param in model.parameters():
        param.requires_grad = False

    if cfg.task == "eval":
        return model

    if cfg.task == 'probing':
        key = "fc" if cfg.model.backbone == "resnet18" else "classifier.1"
        for param in getattr(model, key).parameters():
            param.requires_grad = True

    elif cfg.task == 'fine-tuning':
        for param in model.parameters():
            param.requires_grad = True

    else:
        raise ValueError("Unsupported type. Choose either 'eval', 'probing', or 'fine-tuning'.")

    return model

In [ ]:
#| hide
from omegaconf import OmegaConf
cfg = OmegaConf.load("../cfgs/task_1/resnet/eval.yaml")
cfg.model.ckpt = "../pretrained_backbone/ckpt_resnet18_ep50.pt"

model = init_model(cfg)


/home/ahmed/miniconda3/envs/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ahmed/miniconda3/envs/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded pretrained weights from ../pretrained_backbone/ckpt_resnet18_ep50.pt


/tmp/ipykernel_69040/614623606.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(ckpt_path, map_location='cpu')


In [ ]:
#| hide
backbone = models.resnet18(pretrained=True)
backbone(torch.randn(1,3,256,256)).shape

torch.Size([1, 1000])

In [ ]:
#| hide
backbone

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
#| hide
fe = nn.Sequential(*list(backbone.children())[:-2])

In [ ]:
#| hide
fe(torch.randn(1,3,256,256)).shape

torch.Size([1, 512, 8, 8])

In [ ]:
#| hide
embed_dim = 512
num_heads = 4
mha = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
features = fe(torch.randn(1,3,256,256))
b, c, h, w = features.shape
features = features.view(b, c, h*w).permute(0, 2, 1)  # (b, h*w, c)
attn_output, attn_weights = mha(features, features, features)
attn_output.shape, attn_weights.shape

(torch.Size([1, 64, 512]), torch.Size([1, 64, 64]))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()